# Python标准库系列之Redis模块
http://python.jobbole.com/87305/

## What is redis?

Redis is an open source (BSD licensed), in-memory data structure store, used as database, cache and message broker. It supports data structures such as strings, hashes, lists, sets, sorted sets with range queries, bitmaps, hyperloglogs and geospatial indexes with radius queries. Redis has built-in replication, Lua scripting, LRU eviction, transactions and different levels of on-disk persistence, and provides high availability via Redis Sentinel and automatic partitioning with Redis Cluster.

In [1]:
# 导入模块没报错则安装成功
import redis

## 入门及使用

In [2]:
# 导入模块
import redis
# 连接到Redis服务器
conn = redis.Redis(host='127.0.0.1', port=6379)
# 写入一条数据
conn.set('name','binyang')

True

In [3]:
# 获取一条数据
conn.get('name')

b'binyang'

### 使用连接池连接到Redis
Behind the scenes, redis-py uses a connection pool to manage connections to a Redis server. By default, each Redis instance you create will in turn create its own connection pool. You can override this behavior and use an existing connection pool by passing an already created connection pool instance to the connection_pool argument of the Redis class. You may choose to do this in order to implement client side sharding or have finer grain control of how connections are managed.

In [4]:
pool = redis.ConnectionPool(host='127.0.0.1', port=6379)

In [5]:
conn = redis.Redis(connection_pool=pool)

In [6]:
conn.set('hello','world')

True

In [7]:
conn.get('hello')

b'world'

### 使用套接字连接

In [8]:
r = redis.Redis(unix_socket_path='/tmp/redis.sock')

## API

redis-py提供的API用来操作redis

### String API

#### set(name, value, ex=None, px=None, nx=False, xx=False)
参数	描述
* ex	过期时间（秒）
* px	过期时间（毫秒）
* nx	如果设置为True，则只有name不存在时，当前set操作才执行
* xx	如果设置为True，则只有name存在时，当前set操作才执行

In [9]:
conn.set('k1', 'v1', ex=10, nx=True)

In [10]:
get_k1 =conn.get('k1')
print(get_k1)
import time
time.sleep(10)
print('after 10 sec')
print(get_k1)

b'v1'
after 10 sec
b'v1'


#### setex(name, value, time)
设置过期时间/秒

In [11]:
conn.setex('k','v',1)

True

In [12]:
conn.get('k')

b'v'

#### psetex(name, time_ms, value)

设置过期时间/毫秒

In [13]:
conn.psetex('k',10,'v')

True

In [14]:
conn.get('k')

#### setnx(name, value)

设置值，只有key不存在时，执行设置操作

In [15]:
conn.get('k1')

b'v1'

In [16]:
conn.setnx('k1','v1')

False

In [17]:
conn.get('k1')

b'v1'

In [18]:
conn.setnx('k1','v2')

False

#### mset(*args, **kwargs)

同时设置多个key/value

In [19]:
conn.mset(k1='v1', k2='v2')

True

In [20]:
conn.mset({'k1':'v1', 'k1':'v1'})

True

#### get(name)

获取单个值

#### mget(keys, *args)

获取多个值

In [21]:
conn.mget('k1','k2')

[b'v1', b'v2']

#### getset(name, value)

设置新值并获取原来的值

In [22]:
conn.set('hello', 'world')

True

In [23]:
conn.getset('hello', 'Linux')

b'world'

In [24]:
conn.get('hello')

b'Linux'

#### getrange(key, start, end)

通过索引的方式来获取value的值

In [25]:
conn.set('key','value')

True

In [26]:
conn.getrange('key', 1, 4)

b'alue'

#### setrange(name, offset, value)

根据索引修改value

In [27]:
conn.set('n','123456789')

True

In [28]:
conn.setrange('n', 0, 'a')

9

In [29]:
conn.get('n')

b'a23456789'

#### setbit(name, offset, value)

getbit(name, offset)

获取value对应某一个索引位置对应的值0/1

In [30]:
conn.getbit('k',1)

0

#### bitcount(key, start=None, end=None)

获取key对应二进制中表示1的个数

#### bitop(operation, dest, *keys)

将多个值进行值运算，得出的结果保存到一个新值当中

operation支持AND(并)、OR(或)、NOT(非)、XOR(异或)

In [31]:
conn.mset(n1='abc',n2='cde',n3='adc')

True

In [32]:
conn.bitop('AND','now_key','n1','n2','n3')

3

In [33]:
conn.get('now_key')

b'a`a'

In [34]:
conn.mget('n1','n2','n3')

[b'abc', b'cde', b'adc']

#### strlen(name)

获取value的长度

In [35]:
conn.set('name','bostonma')

True

In [36]:
conn.strlen('name')

8

#### incr(name, amount=1)

对name的value进行自增，如果name不存在则创建，否则自增

In [37]:
conn.set('number',100)
conn.get('number')

b'100'

In [38]:
conn.incr('number')

101

In [39]:
conn.get('number')

b'101'

In [40]:
conn.incr('number')

102

In [41]:
conn.incr('number', 10)

112

#### incrbyfloat(name, amount=1.0)

同上，支持浮点数自增



In [42]:
conn.incrbyfloat('number', 1.5)

113.5

In [43]:
conn.incrbyfloat('number', 1.1)

114.6

#### decr(name, amount=1)

自减，同自增一样，如果进行自减的value不是整数就报错

#### append(key, value)

在value后面追加内容

In [44]:
conn.set('us','bostonma')

True

In [45]:
conn.append('us','2017')

12

In [46]:
conn.get('us')

b'bostonma2017'

## Hash API

#### hset(name, key, value)

设置name的键值对，有则修改，没有则创建

In [47]:
conn.hset('dic','k1','v1')

0

In [48]:
conn.hget('dic','k1')

b'v1'

TBD...

## ListAPI

## SET API

## Ordered set API

## Global API

## 管道

redis-py默认在执行每次请求都会创建（连接池申请连接）和断开（归还连接池）一次连接操作，如果想要在一次请求中指定多个命令，则可以使用pipline实现一次请求指定多个命令，并且默认情况下一次pipline 是原子性操作(MySQL中的事务)。



In [49]:
import redis

In [50]:
pool = redis.ConnectionPool(host='127.0.0.1', port=6379)

In [51]:
r = redis.Redis(connection_pool=pool)

In [52]:
# 创建一个通道支持事务
pipe = r.pipeline(transaction=True)

In [53]:
r.set('hello', 'world')

True

In [54]:
r.set('blog', 'ansheng.me')

True

In [55]:
# 如果在设置上面两个值的过程中出错了，那么这次提交就不会执行
pipe.execute()

[]

## 发布订阅